In [2]:
import numpy as np
from matching_utils import iou

In [3]:
def data_association(dets, trks, threshold=-0.2, algm="greedy"):
    """
    Q1. Assigns detections to tracked object

    dets:       a list of Box3D object
    trks:       a list of Box3D object
    threshold:  only mark a det-trk pair as a match if their iou distance is less than the threshold
    algm:       for extra credit, implement the hungarian algorithm as well

    Returns 3 lists:
        matches, kx2 np array of match indices, i.e. [[det_idx1, trk_idx1], [det_idx2, trk_idx2], ...]
        unmatched_dets, a 1d array of indices of unmatched detections
        unmatched_trks, a 1d array of indices of unmatched trackers
    """
    # Hint: you should use the provided iou(box_a, box_b) function to compute distance/cost between pairs of box3d objects
    # iou() is an implementation of a 3D box IoU

    # --------------------------- Begin your code here ---------------------------------------------
    matches = []
    unmatched_dets = np.ones(len(dets))
    unmatched_trks = np.ones(len(trks))

    gious = np.zeros((len(dets), len(trks)))
    for i in range(len(dets)):
        for j in range(len(trks)):
            gious[i, j] = iou(dets[i], trks[j])

    while len(dets) > 0 and len(trks) > 0 and np.max(gious) >= threshold:
        idx = np.unravel_index(np.argmax(gious), gious.shape)
        unmatched_dets[idx[0]] = 0  # mask as selected
        unmatched_trks[idx[1]] = 0  # mask as selected
        matches.append([idx[0], idx[1]])

        gious[idx[0], :] = -1.0  # mask all selected matches
        gious[:, idx[1]] = -1.0

    unmatched_dets = np.array(*np.where(unmatched_dets > 0)).astype(int)
    unmatched_trks = np.array(*np.where(unmatched_trks > 0)).astype(int)

    matches = np.array(matches)
    # --------------------------- End your code here   ---------------------------------------------

    return matches, unmatched_dets, unmatched_trks

In [4]:
def get_index(cost):
    if len(cost) == 1:
        if cost == 0:
            return np.array([0, 0], dtype=int).reshape(1, -1), True
        else:
            return [], False

    for i in np.argwhere(cost[:, 0] == 0):
        rows = list(range(cost.shape[0]))
        rows.remove(i)
        next_cost = cost[:, 1:].take(rows, axis=0)
        matches, success = get_index(next_cost)
        if success:
            matches[:, -1] += 1
            matches[np.argwhere(matches[:, 0] >= i), 0] += 1
            matches = np.append(matches, [[int(i), 0]], axis=0)
            return matches[matches[:, 0].argsort()], True

    return [], False

In [22]:
def get_rows_columns(cost, debug=False):
    tmp_cost = cost.copy()
    marked_rows = []
    marked_columns = []

    while np.min(tmp_cost) == 0:
        column_zeros = tmp_cost.shape[0] - np.count_nonzero(tmp_cost, axis=0)
        row_zeros = tmp_cost.shape[1] - np.count_nonzero(tmp_cost, axis=1)

        column_vs_row_zeros = np.array(np.meshgrid(column_zeros, row_zeros)).transpose(
            [1, 2, 0]
        )
        column_plus_row_zeros = np.sum(column_vs_row_zeros, axis=-1) - (tmp_cost == 0)

        if debug:
            print("marked_rows =", marked_rows)
            print("marked_columns =", marked_columns)
            print("tmp_cost =", tmp_cost)
            print("column_plus_row_zeros =", column_plus_row_zeros)

        pivot_point = np.unravel_index(
            np.argmax(column_plus_row_zeros), column_plus_row_zeros.shape
        )

        if np.min(tmp_cost[pivot_point[0]]) == 0:
            tmp_cost[pivot_point[0]] = 1
            marked_rows.append(pivot_point[0])

        if np.min(tmp_cost[:, pivot_point[1]]) == 0:
            tmp_cost[:, pivot_point[1]] = 1
            marked_columns.append(pivot_point[1])

    return np.sort(np.array(marked_rows)), np.sort(np.array(marked_columns))

In [27]:
dets = np.zeros(3)
trks = np.zeros(9)
threshold = -0.2

matches = []

# size of the gious and cost matrix, might need padding
mat_size = max(len(dets), len(trks))

print(mat_size)

gious = np.random.uniform(-3, -1, size=(mat_size, mat_size))

gious[:3, :] = np.array(
    [
        [
            0.86519114,
            -0.77648173,
            -0.8221538,
            -0.92700256,
            -0.93348205,
            -0.84591054,
            -0.87322746,
            -0.89241218,
            -0.87287956,
        ],
        [
            -0.83221675,
            -0.84459791,
            0.79870613,
            -0.85421317,
            -0.93761682,
            -0.84378765,
            -0.87912881,
            -0.88578027,
            -0.7336418,
        ],
        [
            -0.86688221,
            -0.91506194,
            -0.71412028,
            -0.61980565,
            -0.90773283,
            -0.81540975,
            -0.85143587,
            -0.88793782,
            0.58528197,
        ],
    ]
)
cost = 1 - gious

print(cost)

# Step 1, Switch Step 1 and Step 2 for optimization purpose
cost = cost.T
cost -= cost.min(axis=1).reshape(-1, 1)
cost = cost.T

tmp_matches, success = get_index(cost)

if not success:
    # Step 2
    cost -= cost.min(axis=1).reshape(-1, 1)

    tmp_matches, success = get_index(cost)

while not success:
    # Step 3
    marked_rows, marked_columns = get_rows_columns(cost)

    # Step 4

    unmarked_rows = np.ones(cost.shape[0])
    unmarked_rows[marked_rows] = 0
    unmarked_rows = np.argwhere(unmarked_rows).reshape(-1, 1)  # strange hack

    unmarked_columns = np.ones(cost.shape[0])
    unmarked_columns[marked_columns] = 0
    unmarked_columns = np.argwhere(unmarked_columns).reshape(-1)

    unmarked_chunks = cost[unmarked_rows, unmarked_columns]

    unmarked_chunks_min = np.min(unmarked_chunks)
    cost[unmarked_rows, unmarked_columns] -= unmarked_chunks_min
    cost[marked_rows.reshape(-1, 1), marked_columns] += unmarked_chunks_min

    # if (len(marked_rows) + len(marked_columns)) > mat_size:
    #     # marked_rows, marked_columns = get_rows_columns(cost, debug = True)
    #     print("cost =\n", cost)
    #     print("marked_rows =", marked_rows)
    #     print("marked_columns =", marked_columns)
    #     print("#rows + columns =", len(marked_rows) + len(marked_columns))
    # Step 5
    tmp_matches, success = get_index(cost)

for i in range(len(tmp_matches)):
    index = tuple(tmp_matches[i])
    if gious[index] >= threshold:
        matches.append(index)
matches = np.array(matches)

print(matches)

unmatched_dets = np.ones(mat_size)
unmatched_dets[matches[:, 0]] = 0
unmatched_dets = np.argwhere(unmatched_dets).reshape(-1)
unmatched_dets = unmatched_dets[unmatched_dets < len(dets)]
unmatched_dets = np.array([*unmatched_dets]).astype(int)
print(unmatched_dets, unmatched_dets.dtype)

unmatched_trks = np.ones(mat_size)
unmatched_trks[matches[:, 1]] = 0
unmatched_trks = np.argwhere(unmatched_trks).reshape(-1)
unmatched_trks = unmatched_trks[unmatched_trks < len(trks)]
unmatched_trks = np.array([*unmatched_trks]).astype(int)
print(unmatched_trks, unmatched_trks.dtype)

9
[[0.13480886 1.77648173 1.8221538  1.92700256 1.93348205 1.84591054
  1.87322746 1.89241218 1.87287956]
 [1.83221675 1.84459791 0.20129387 1.85421317 1.93761682 1.84378765
  1.87912881 1.88578027 1.7336418 ]
 [1.86688221 1.91506194 1.71412028 1.61980565 1.90773283 1.81540975
  1.85143587 1.88793782 0.41471803]
 [2.57657562 2.72066312 3.91256854 2.15641094 2.85947984 3.34421231
  3.87466547 3.13008192 2.6401411 ]
 [3.12215212 3.50017702 2.17903962 3.99981339 3.24512296 3.42114529
  3.96065223 3.99270621 2.82826631]
 [3.670616   3.79320902 3.17919279 2.92393132 2.61761332 2.05910196
  3.09912136 2.14790788 3.31962823]
 [3.09403699 2.5309524  2.99726372 3.83940606 3.03462295 2.69748859
  2.75389517 3.54667853 3.6903738 ]
 [2.31672992 2.23470269 3.9459418  2.92133995 3.13328244 3.87151211
  3.27719473 3.83675922 2.76330057]
 [2.81483603 2.09013651 3.08517824 3.87956579 3.12813079 3.6646424
  3.40135322 2.6807542  3.86141004]]
[[0 0]
 [1 2]
 [2 8]]
[] int32
[1 3 4 5 6 7] int32


In [25]:
np.random.uniform(-3, -1, size=(mat_size, mat_size))

array([[-1.47901349, -2.0639314 , -2.18489525, -2.20617812, -1.00616996,
        -1.2440079 , -2.55731821, -1.33464666, -1.5461023 ],
       [-2.99989109, -2.48157183, -1.7657035 , -1.42675   , -1.31113392,
        -1.77039475, -2.62282393, -2.49263864, -1.4448513 ],
       [-2.81517201, -1.95869474, -1.55384892, -2.88116426, -2.6579116 ,
        -2.21302718, -2.3598021 , -1.51443981, -1.32543826],
       [-1.28678793, -2.5937415 , -1.63447814, -2.4084412 , -2.61080576,
        -1.3097032 , -1.07576404, -2.47416   , -2.76141313],
       [-2.51079002, -1.66937392, -1.46912871, -2.91689769, -1.06884026,
        -1.75771296, -2.5136248 , -2.3177774 , -1.18665156],
       [-2.74859239, -1.57571709, -2.90024927, -2.38085075, -2.78954417,
        -1.34146081, -2.80930441, -1.22658407, -1.6637173 ],
       [-1.65529258, -2.62722493, -2.51232054, -2.7404096 , -1.37133557,
        -1.5239046 , -2.43968279, -1.06243095, -1.77442682],
       [-1.13882967, -1.34966113, -2.02897118, -1.76148814, -1

In [188]:
# cost = np.array([[0, 1, 1, 1, 1],
#                  [1, 1, 0, 0, 1],
#                  [1, 1, 0, 1, 0],
#                  [1, 0, 1, 1, 1],
#                  [0, 1, 1, 0, 1]])

cost = np.array(
    [
        [0, 1, 1, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 0, 0, 0],
        [1, 0, 1, 1, 1],
        [0, 1, 1, 0, 1],
    ]
)

# cost = np.array([[0, 1, 0, 1, 1],
#                  [1, 1, 0, 1, 1],
#                  [1, 0, 0, 0, 1],
#                  [1, 1, 0, 1, 1],
#                  [1, 0, 0, 1, 0]])

print(cost)

marked_rows, marked_columns = get_rows_columns(cost)

print("marked_rows =", marked_rows)
print("marked_columns =", marked_columns)

unmarked_rows = np.ones(cost.shape[0])
unmarked_rows[marked_rows] = 0
unmarked_rows = np.argwhere(unmarked_rows).reshape(-1)

unmarked_columns = np.ones(cost.shape[0])
unmarked_columns[marked_columns] = 0
unmarked_columns = np.argwhere(unmarked_columns).reshape(-1)

unmarked_chunks = cost[unmarked_rows.reshape(-1, 1), unmarked_columns]
intersection_points = cost[marked_rows.reshape(-1, 1), marked_columns]

unmarked_chunks_min = np.min(unmarked_chunks)
cost[unmarked_rows.reshape(-1, 1), unmarked_columns] -= unmarked_chunks_min
cost[marked_rows.reshape(-1, 1), marked_columns] += unmarked_chunks_min

print(intersection_points, unmarked_chunks)
print(cost)

[[0 1 1 1 1]
 [1 1 1 0 1]
 [1 1 0 0 0]
 [1 0 1 1 1]
 [0 1 1 0 1]]
marked_rows = [2 3]
marked_columns = [0 3]
[[1 0]
 [1 1]] [[1 1 1]
 [1 1 1]
 [1 1 1]]
[[0 0 0 1 0]
 [1 0 0 0 0]
 [2 1 0 1 0]
 [2 0 1 2 1]
 [0 0 0 0 0]]
